# MASK-Maintaining Data Privacy in Association Rule Mining

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
link)

In [40]:

test_ds_path = ".\..\Datasets\store_data.csv"

with open(test_ds_path, "r") as f:

    lines = f.readlines()
    inventory = list()
    tuples = list()

    for line in lines:
        transaction = line.strip().split(",")
        #print(transaction)
        tuples.append(transaction)
        for element in transaction:
            if element.strip().replace(' ','_').replace('&','and') not in inventory:
                inventory.append(element.strip().replace(' ','_').replace('&','and'))

In [ ]:
print(len(inventory))

In [ ]:

print(inventory)
inventory.pop()
inventory.sort()
print(inventory)

In [ ]:
print(len(inventory))

In [44]:
import pandas as pd
import numpy as np
import math
import random
from pandas import DataFrame


In [45]:
test_dataset = pd.DataFrame(
    [[1 if item in row else 0 for item in inventory] for row in tuples],
    columns=inventory
)

In [ ]:
test_dataset.head(3)

In [ ]:
test_dataset.columns

In [ ]:
type(test_dataset.columns)

In [ ]:
for tuple in test_dataset.itertuples():
    print(tuple)
    print(int( getattr(tuple, 'almonds')))
    print(type(tuple))
    break

In [50]:
from numpy import float64
class Rule:
    def __init__(self,itemset: list | dict,support: float = 0, confidence: float = 0):
        self.itemset = itemset
        self.support = support
        self.confidence = confidence

    def __str__(self):
        return f"Rule('{self.itemset}', '{self.support}')"
    
    def __repr__(self):
        return f"Rule('{self.itemset}', '{self.support}')"
    
    def __eq__(self, other):
        if isinstance(other, Rule):
            return set(self.itemset) == set(other.itemset)
        return False
    
    def __iter__(self):
        return iter([self.itemset, self.support, self.confidence])
    

class AprioriRule(Rule):
    def __init__(self,itemset: list | dict,support: float64 = 0):
        super().__init__(itemset,support)

In [51]:

from pandas import DataFrame


def support(T: DataFrame ,X_U_Y: list | dict):
    '''
    Parameters:
    T (dataframe)
    X_U_Y (list | dict) : name of the attributes considered (X U Y)
    Return:
    float: support of the attributes in the dataset
    '''
    for attribute in X_U_Y:
        if attribute not in T.columns:
             return ValueError
    count = 0
    for tuple in T.itertuples(False):
        contained = True
        for attribute in X_U_Y:
            try: 
                if getattr(tuple,attribute) == 0:
                    contained = False
                    break
            except AttributeError:
                contained = False,
                break

        if contained:
            count += 1
    return count/len(T)
    

In [ ]:
support(test_dataset,['herb_and_pepper'])

In [53]:

def confidence(T: DataFrame, X: list | dict, Y: list | dict):
    '''
    Parameters:
    T (dataframe)
    X ( list | dict): X part of X ==> Y rule
    X (list | dict): Y part of X ==> Y rule
    '''
    if len(X)+len(Y) > len(T.columns):
        return ValueError
    for item in X:
        if Y.__contains__(item):
            return ValueError # XY = empty set
    
    countX=0
    countY=0
    for tuple in T.itertuples():
        containedX = True
        for attribute in X:
            if getattr(tuple,attribute) == 0:
                containedX = False
                break
        if containedX:
            countX += 1
            containedY=True
            for attribute in Y:
                if attribute == '':
                    containedY = False
                    break
                if getattr(tuple,attribute) == 0:
                    containedY = False
                    break
            if containedY:
                countY += 1
    return countY/countX    

In [ ]:
confidence(test_dataset,['spaghetti','champagne'],['cookies'])
#spaghetti,champagne,cookies

In [55]:
def Apriori(items, dataset, min_sup, levels: int = None):
    if levels is None:
        levels = len(items)
    '''
    rules[0] = empty set
    rules[1] = rules of length 1
    rules[2] = rules of length 2


    rules[len(items)] = items
    ...
    '''
    rules = [
        [],
        [ AprioriRule([item]) for item in items ]
    ]



    # iterate over all the possible rules length from 1 to len(items)
    for i in range(1, levels+1):
        print(f"Apriori Level: {i}")
        # remove all the rules in rules[i]
        # that don't have a support of at least min_sup

        #print(f"RULES[{i}] BEFORE", rules[i])
        for j in range(len(rules[i])-1,-1,-1):
            sup = support(dataset,rules[i][j].itemset)
            if sup >= min_sup:
                rules[i][j].support = sup
            else:
                rules[i].remove(rules[i][j])
    
        '''
        print(f"RULES[{i}] SUPPORT", [
            support(dataset, rule)
            for rule in rules[i]
        ]
        )'''


        #print(f"RULES[{i}] AFTER", rules[i])

        if len(rules[i]) == 0:
            break


        # generate all the possibile 
        # rules with i+1 elements
        rules.append([]) # create the element rules[i+1]

        #print("RULE[i]", rules[i])
        #print("RULE[i+1]", rules[i+1])
        for rule in rules[i]:
            for item in items:

                # skip if item is already inside the rule
                if item in rule.itemset:
                    continue

                rules[i+1].append ( AprioriRule(rule.itemset + [item]))
                # create a new rule composed of rule + [item]


    return rules

In [ ]:
support_vector = [[item,support(test_dataset,[item])] for item in inventory]
print(support_vector)
mean_support = np.mean([sublist[1] for sublist in support_vector])
print(mean_support)


In [57]:


def MASK_Distortion(dataset: DataFrame, p: float):
    '''
    MASK
    
    choose a probability p

    2 event
    P(x = true) -> 1-p % -> we add or remove an element in the transaction
    P(x = false) -> p% -> the transaction remains the same
    '''
    distorted_dataset = dataset.copy(deep=True)
    for i in range(0,len(distorted_dataset)):
        for column in distorted_dataset.columns:
            event = random.random()
            if event > p:
                distorted_dataset.loc[i, column] = int(not dataset.loc[i, column])

    return distorted_dataset

        



In [59]:
p = 0.9
distorted_test_dataset = MASK_Distortion(test_dataset,p)

In [60]:
def hammingDistanceBitwise(a:int,b:int):
    return (a^b).bit_count()

In [ ]:
hammingDistanceBitwise(int(1),int(2))

In [ ]:


M = np.diag([math.pow(2,3) for i in range(0,3)])
print(M)

In [63]:

def computeM(size: int, p: float):
    max_exp = int(math.log2(size))
    M = np.diag([math.pow(p,max_exp) for i in range(size)])
    for i in range(0,size):
        for j in range(i+1,size):
            difference = hammingDistanceBitwise(i,j)
            M[i][j] = math.pow(1-p,difference)*math.pow(p,max_exp-difference)

    for i in range(1,size):
        for j in range(i,-1,-1):
            M[i][j] = M[j][i]

    return M


In [ ]:
M_test = computeM(int(math.pow(2,2)),0.1)
print(M_test)
print(sum(M_test[0]))

In [66]:
def vectormatrixProdMod(linC_D,matrix):
    size = matrix.shape[0]
    row = matrix[size-1]
    sum:float = 0.0
    for j in range(len(linC_D)):
        index = int(math.pow(2,j))-1
        #print(str(row[index])+" * "+str(linC_D[j]))
        sum += row[index]*linC_D[j]
    return sum

In [ ]:
from numpy.linalg import inv

M_test = inv(computeM(2**2,p))
print(M_test)
print(sum(M_test[3]))

In [70]:
class MASKRule(Rule):
    def __init__(self,itemset: list | dict,support: float = 0):
        super().__init__(itemset,support)
        self.counters = np.zeros(len(itemset)+1)

In [71]:
def MASK_Support(linC_D,db_cardinality,M_inv = None):
    if M_inv is None:
        M_inv = inv(
            computeM(
                size=int(math.pow(2,len(linC_D)-1)),
                p=p
            )
        )

    C_T_11 = vectormatrixProdMod(linC_D,M_inv)

    return C_T_11/db_cardinality
    
    

In [106]:
import itertools

def MASK_Rule_Mining(dataset: DataFrame, p: float, min_sup: float,levels: int = None):
    if levels is None:
        levels = len(dataset.columns)
    rules = [
        [],
    ]
    frequent_itemsets = [[]]
    infrequent_itemsets = [[]]

    for i in range(1,levels+1):
        print(f"Mask Rule Mining level: {i}")

        combinations = list(itertools.combinations(dataset.columns,i))
        rule_i = [
            MASKRule(sorted(list(c)))for c in combinations]
        rules.append(rule_i)

        frequent_itemsets.append([])
        infrequent_itemsets.append([])


        for tuple in dataset.itertuples():

            item_list = []
            complement_list=[]

            for item in dataset.columns:
                if getattr(tuple,item) == 1 and item not in infrequent_itemsets[i-1]:
                    item_list.append(item)
            for item in frequent_itemsets[i-1]:
                if item not in item_list:
                    complement_list.append(item)

           
            for rule in rules[i]:
                bit_counter = 0
                for item in rule.itemset:
                    if item in item_list:
                        bit_counter += 1
                
                rule.counters[bit_counter]+=1
            
        
        

        for j in range(len(rules[i])-1,-1,-1):

            size = int(math.pow(2,i))
            M_inv = inv(
                computeM(size,p)
            )

            sup = MASK_Support(rules[i][j].counters,len(dataset),M_inv)

            if sup >= min_sup:
                rules[i][j].support = sup
                for item in rules[i][j].itemset:
                    if item not in frequent_itemsets[i]:
                        frequent_itemsets[i].append(item)

            else:
                rules[i].remove(rules[i][j])
                for item in rule.itemset:
                    if item not in infrequent_itemsets[i]:
                        infrequent_itemsets[i].append(item)
            
        if len(rules[i]) == 0:
            break
    
    return rules

In [ ]:
reduced_inventory = inventory[0:50]
print(reduced_inventory)

In [ ]:
reduced_test_ds = test_dataset.iloc[:, 0:50]
print(reduced_test_ds)

In [ ]:
support_vector_reduced = [[item,support(reduced_test_ds,[item])] for item in reduced_inventory]
print(support_vector_reduced)
mean_reduced_support = np.mean([sublist[1] for sublist in support_vector_reduced])
print(mean_reduced_support)

In [101]:
p = 0.9
reduced_distorted_test_ds = MASK_Distortion(reduced_test_ds,p)

In [ ]:
min_sup = 0.01
rules = Apriori(reduced_inventory,reduced_test_ds,min_sup)
print(rules)

In [ ]:
MASK_rules = MASK_Rule_Mining(reduced_distorted_test_ds,p,min_sup)
#print(MASK_rules)

In [ ]:
print(MASK_rules)

## Evaluating performance of the algorithm
### Metrics
* Support Error:
    $\rho = \frac{1}{|f|}\sum_{f}^{}\frac{|recSup_f - actSup_f|}{actSup_f}*100 $
* Identity error:
    $\sigma^+$ = percentage of false positive
    $\sigma^-$ = percentage of false negative

    $\sigma^+ = \frac{|R-F|}{|F|}*100$      $\sigma^- = \frac{|F-R|}{|F|}*100$


In [82]:
def support_error(AprioriRuleslevel,MASKRuleslevel):
    
    sum = 0
    cnt = 0

    for apriorirule in AprioriRuleslevel:
        if apriorirule is not None and apriorirule in MASKRuleslevel:
            cnt += 1
            index = MASKRuleslevel.index(apriorirule)
            
            rec_sup = MASKRuleslevel[index].support
            act_sup = apriorirule.support
            
            sum += abs(rec_sup-act_sup)/act_sup
    if cnt != 0:
        return (sum/cnt)*100
    else:
        return 0
    


In [110]:
def identity_error(AprioriRulesLevel,MASKRulesLevel):

    false_positive_cnt = 0
    false_negative_cnt = 0

    for rule in AprioriRulesLevel:
        if rule not in MASKRulesLevel:
            false_positive_cnt += 1
    
    for rule in MASKRulesLevel:
        if rule not in AprioriRulesLevel:
            false_negative_cnt += 1
    
    F = len(AprioriRulesLevel)

    if F != 0:
        false_positive = false_positive_cnt/F
        false_negative = false_negative_cnt/F
        return false_positive, false_negative
    else: return 0,0

    

In [ ]:
for i in range(1,len(rules)):
    print(support_error(rules[i],MASK_rules[i]))

In [ ]:
for i in range(1,len(rules)):
    print(identity_error(rules[i],MASK_rules[i]))

## Evaluating MASK's privacy
### Metrics:
* Reconstruction Probability:
